<a href="https://colab.research.google.com/github/GuysBarash/Genetic-programing-with-DEAP/blob/master/evolutionarty_GP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# !pip install DEAP
pass

In [0]:
import random
import operator

import numpy
import pandas as pd

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp
from datetime import datetime
from IPython.display import display, HTML
import matplotlib.pyplot as plt

# Generate Data

In [0]:
MUX_SELECT_LINES = 3
MUX_IN_LINES = 2 ** MUX_SELECT_LINES
MUX_TOTAL_LINES = MUX_SELECT_LINES + MUX_IN_LINES

def if_then_else(condition, out1, out2):
    return out1 if condition else out2

# input : [A0 A1 A2 D0 D1 D2 D3 D4 D5 D6 D7] for a 8-3 mux
inputs = [[0] * MUX_TOTAL_LINES for i in range(2 ** MUX_TOTAL_LINES)]
outputs = [None] * (2 ** MUX_TOTAL_LINES)

for i in range(2 ** MUX_TOTAL_LINES):
    value = i
    divisor = 2 ** MUX_TOTAL_LINES
    # Fill the input bits
    for j in range(MUX_TOTAL_LINES):
        divisor /= 2
        if value >= divisor:
            inputs[i][j] = 1
            value -= divisor

    # Determine the corresponding output
    indexOutput = MUX_SELECT_LINES
    for j, k in enumerate(inputs[i][:MUX_SELECT_LINES]):
        indexOutput += k * 2 ** j
    outputs[i] = inputs[i][indexOutput]

datacols = [f'bx{idx}' for idx in range(MUX_TOTAL_LINES)]
labelcol = 'LABEL'
datadf = pd.DataFrame(columns=datacols,
                      index=range(2 ** MUX_TOTAL_LINES),
                      data=inputs)
datadf[labelcol] = outputs

In [15]:
top_n = 14
display(datadf.head(14))
print("")
print(f"Displaying top {top_n} out of {len(datadf)}")

,bx0,bx1,bx2,bx3,bx4,bx5,bx6,bx7,bx8,bx9,bx10,LABEL
0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,1,1,0
4,0,0,0,0,0,0,0,0,1,0,0,0
5,0,0,0,0,0,0,0,0,1,0,1,0
6,0,0,0,0,0,0,0,0,1,1,0,0
7,0,0,0,0,0,0,0,0,1,1,1,0
8,0,0,0,0,0,0,0,1,0,0,0,0
9,0,0,0,0,0,0,0,1,0,0,1,0



Displaying top 14 out of 2048


# Defining evolution parameters

Legal nodes in tree

In [0]:
pset = gp.PrimitiveSet("MAIN", MUX_TOTAL_LINES, "IN")
pset.addPrimitive(operator.and_, 2)
pset.addPrimitive(operator.or_, 2)
pset.addPrimitive(operator.not_, 1)
pset.addPrimitive(if_then_else, 3)
pset.addTerminal(1)
pset.addTerminal(0)

Evolution parameters

In [0]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genFull, pset=pset, min_=2, max_=4)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

Support functions

In [0]:
evaluation_counter = 0

def individual_to_function(individual):
    return toolbox.compile(expr=individual)


def apply_function_to_df(func, dataset=None):
  if dataset is None:
    dataset =  datadf[datacols]
  results = datadf[datacols].apply(lambda r: func(*r.values), axis=1)
  return results


def evalMultiplexer(individual):
    global evaluation_counter
    evaluation_counter += 1
    func = individual_to_function(individual)
    results = apply_function_to_df(func)
    fitness = (results == datadf[labelcol]).sum()
    max_score = len(datadf)
    fitness_score = fitness / max_score
    return fitness_score,

In [0]:
toolbox.register("evaluate", evalMultiplexer)
toolbox.register("select", tools.selTournament, tournsize=7)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genGrow, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

# Evolution part

In [0]:
toolbox.register("evaluate", evalMultiplexer)
toolbox.register("select", tools.selTournament, tournsize=7)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genGrow, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

In [21]:
    # random.seed(10)
    pop = toolbox.population(n=25)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    print("Starting evolution.")
    start_time = datetime.now()
    algorithms.eaSimple(pop, toolbox, cxpb=0.8, mutpb=0.1, ngen=25, stats=stats, halloffame=hof)
    duration = datetime.now() - start_time
    print("Run concluded.")
    print("Evaluations commited: {}".format(evaluation_counter))
    print("Run time: {}".format(duration))
    winner_creature = hof.items[0]
    winner_function = individual_to_function(winner_creature)

Starting evolution.
gen	nevals	avg     	std      	min   	max     
0  	25    	0.504297	0.0421875	0.4375	0.578125
1  	21    	0.549219	0.0195156	0.5   	0.578125
2  	20    	0.561094	0.0248747	0.46875	0.59375 
3  	21    	0.574219	0.0126938	0.546875	0.59375 
4  	18    	0.576094	0.0192992	0.53125 	0.59375 
5  	20    	0.584219	0.0149086	0.53125 	0.59375 
6  	20    	0.584375	0.0182217	0.53125 	0.59375 
7  	22    	0.580781	0.0190958	0.53125 	0.609375
8  	18    	0.585469	0.0207383	0.5     	0.609375
9  	22    	0.587656	0.0204347	0.515625	0.625   
10 	22    	0.59375 	0.0274662	0.53125 	0.625   
11 	16    	0.603125	0.0279508	0.546875	0.625   
12 	18    	0.608281	0.0259469	0.5625  	0.625   
13 	23    	0.594688	0.0317399	0.53125 	0.625   
14 	18    	0.600781	0.032476 	0.5     	0.625   
15 	16    	0.61625 	0.0212316	0.5625  	0.65625 
16 	20    	0.603125	0.0296464	0.546875	0.65625 
17 	18    	0.610938	0.0402628	0.5     	0.65625 
18 	20    	0.624687	0.0295209	0.5625  	0.65625 
19 	22    	0.612187	0.03569

# Analyze results

In [0]:
predictions = apply_function_to_df(winner_function)
true_results = datadf[labelcol]

In [0]:
from sklearn.metrics import plot_confusion_matrix

# Plot non-normalized confusion matrix
titles_options = [("Confusion matrix, without normalization", None),
                  ("Normalized confusion matrix", 'true')]
for title, normalize in titles_options:
    disp = plot_confusion_matrix(classifier, X_test, y_test,
                                 display_labels=class_names,
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

plt.show()